In [1]:
import pandas as pd

csv_path = "synthetic_wsht_weather_migraine_prob_600days_hourly_FIXED.csv"
df = pd.read_csv(csv_path)

print(df.columns)
df.head()


Index(['timestamp', 'workload_0_10', 'stress_0_10', 'hrv_rmssd_ms',
       'pressure_hpa', 'humidity_pct', 'temperature_c',
       'migraine_prob_next_hour', 'hour', 'dayofweek', 'is_weekday'],
      dtype='object')


,timestamp,workload_0_10,stress_0_10,hrv_rmssd_ms,pressure_hpa,humidity_pct,temperature_c,migraine_prob_next_hour,hour,dayofweek,is_weekday
0,2023-01-01 00:00:00,3.37,4.96,44.29,1012.8,50.2,13.2,NaN,0,6,0
1,2023-01-01 01:00:00,2.78,3.76,48.36,1012.3,57.3,12.0,0.0187,1,6,0
2,2023-01-01 02:00:00,2.70,3.71,49.09,1012.6,58.6,11.5,0.0121,2,6,0
3,2023-01-01 03:00:00,2.97,3.77,51.59,1012.1,60.3,9.6,0.0038,3,6,0
4,2023-01-01 04:00:00,2.27,1.68,58.80,1011.2,61.1,8.8,0.0090,4,6,0


In [ ]:
if "timestamp" in df.columns:
    df["timestamp"] = pd.to_datetime(df["timestamp"])
df = df.reset_index(drop=True)
df["row_idx"] = df.index


In [ ]:
prob_col = "migraine_prob_next_hour"  # change if needed

def find_index_for_target_prob(df, target, prob_col, window_size=24):
    # exclude the first few rows where we can't get a full window
    df_valid = df[df.index >= (window_size - 1)].copy()
    # also exclude any NaNs
    df_valid = df_valid[df_valid[prob_col].notna()]

    # compute absolute difference from target
    df_valid["diff"] = (df_valid[prob_col] - target).abs()

    # pick the row with minimum difference
    best_row = df_valid.loc[df_valid["diff"].idxmin()]

    i = int(best_row["row_idx"])
    actual_p = float(best_row[prob_col])
    return i, actual_p, best_row
